In [1]:
import io
import numpy
from PIL import Image
import aiohttp
import asyncio
import pyproj
from lxml import etree

In [2]:
#XML DATA
_FILE_PATH = 'wmts.xml'
_DEF_NS = {"foo": "http://www.opengis.net/wmts/1.0","ows":"http://www.opengis.net/ows/1.1"}

In [3]:
base_url = "https://wxs.ign.fr/an7nvfzojv5wa96dsga5nk8w/geoportail/wmts"
_headers = {
        'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'en-US,en;q=0.5',
        'Connection': 'keep-alive',
        'Host': 'wxs.ign.fr',
        'Referer': 'http://www.geoportail.gouv.fr/swf/geoportal-visu-1.3.2.swf',
        'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:22.0) Gecko/20100101 Firefox/22.0',
}
default_values = {
    'layer' : 'GEOGRAPHICALGRIDSYSTEMS.MAPS.SCAN25TOUR.CV',
    'style' : 'normal',
    'tilematrixset' : 'PM',
    'Service' : 'WMTS',
    'Request' : 'GetTile',
    'Version' : '1.0.0',
    'Format': 'image/jpeg',
    'TileMatrix' : 15
    }
default_values_save = {
    'layer' : 'GEOGRAPHICALGRIDSYSTEMS.MAPS.SCAN-EXPRESS.STANDARD',
    'style' : 'normal',
    'tilematrixset' : 'PM',
    'Service' : 'WMTS',
    'Request' : 'GetTile',
    'Version' : '1.0.0',
    'Format': 'image/jpeg',
    'TileMatrix' : 16
    }
#to get more info : https://wxs.ign.fr/an7nvfzojv5wa96dsga5nk8w/geoportail/wmts?Service=WMTS&Request=GetCapabilities&Version=1.0.0&Layer=GEOGRAPHICALGRIDSYSTEMS.MAPS.SCAN25TOUR.CV

In [4]:
def get_scale_denom_info(file):
    """Return dict with tile matrix set identifier, zoom(scale), scale denom(meters/pixel)"""
    data = {}
    tree = etree.parse(_FILE_PATH)
    
    TileMatrixSets = tree.xpath('/foo:Capabilities/foo:Contents/foo:TileMatrixSet', namespaces = _DEF_NS)
    for tilematrixset in TileMatrixSets:
        identifier = tilematrixset.find('ows:Identifier', namespaces = _DEF_NS).text
        supportedcrs = tilematrixset.find('ows:SupportedCRS', namespaces = _DEF_NS).text
        data[identifier] = {}
        data[identifier]['supportedcrs'] = supportedcrs
        TileMatrixes = tilematrixset.findall('foo:TileMatrix', namespaces = _DEF_NS)
        for tilematrix in TileMatrixes:
            scale = tilematrix.find('ows:Identifier', namespaces = _DEF_NS).text
            #Dénominateur d'échelle = résolution / taille pixel ; taille de pixel arbitraire = 0.00028 m
            scale_denom = float(tilematrix.find('foo:ScaleDenominator', namespaces = _DEF_NS).text) * 0.00028 
            data[identifier][scale]=scale_denom
            str_ref = tilematrix.find('foo:TopLeftCorner', namespaces = _DEF_NS).text
            (data[identifier]['X_ref'],data[identifier]['Y_ref']) =  map(float, str_ref.split(" "))
    return data

In [5]:
matrix_set_data = get_scale_denom_info(_FILE_PATH)

In [21]:
def convert_coord(lat_init, long_init, matrix_data, zoom, projection ):
    """convert decimals coords into Web Mercator coords"""
    #See https://geoservices.ign.fr/documentation/geoservices/wmts.html 
    #data
    res_WMS = [156543.0339280410,78271.5169640205,39135.7584820102,19567.8792410051,9783.9396205026,4891.9698102513,2445.9849051256,1222.9924525628,611.4962262814,305.7481131407,152.8740565704,76.4370282852,38.2185141426,19.1092570713,9.5546285356,4.7773142678,2.3886571339,1.1943285670,0.5971642835,0.2985821417,0.1492910709,0.0746455354]
    res_L93 = [104579.2245498940,52277.5323537905,26135.4870785954,13066.8913818000,6533.2286041135,3266.5595244627,1633.2660045974,816.6295549860,408.3139146768,204.1567415109,102.0783167832,51.0391448966,25.5195690743,12.7597836936,6.3798916360,3.1899457653,1.5949728695,0.7974864315,0.3987432149,0.1993716073,0.0996858037,0.0498429018]
    
    long_lat = pyproj.Proj(init='epsg:4326')
    
    final_proj = pyproj.Proj(init=matrix_data[projection]['supportedcrs'])
    tile_res = matrix_data[projection][str(zoom)]*256 #256 pixel per tile
    
    (x, y) = pyproj.transform(long_lat, final_proj, long_init, lat_init)

    # Get the top left corner
    (x0, y0) = (matrix_data[projection]['X_ref'], matrix_data[projection]['Y_ref'])
    
    xf = int((x - x0) / tile_res)
    yf = int((y0 - y) / tile_res) 
    print("Tile Rez: " + str(tile_res)+ " X,Y: "+str(xf)+","+str(yf))
        
    return(xf, yf)

In [33]:
def get_map(lat, long, row_number, col_number, zoom = 15, projection = "PM"):
    
    contenant = numpy.empty([row_number,col_number,256,256,3], dtype = 'uint8')

    async def put_tile(session, Row, Col, zoom):
        start_col, start_row = convert_coord(lat, long, matrix_set_data, zoom, projection)
        data = await get_tile(session, start_row+Row, start_col+Col, zoom)
        contenant[Row][Col] = Image.open(io.BytesIO(data))
        
    async def get_tile(session, Row, Col, zoom):
        _values = default_values
        _values['TileCol'] = Col
        _values['TileRow'] = Row
        _values['TileMatrix'] = zoom

        async with session.request('GET', base_url, params = _values) as resp:
            print(resp.status)
            assert resp.status == 200
            return await  resp.read()
        
    async def main(loop):
        tasks = []
        async with aiohttp.ClientSession(loop=loop, headers = _headers ) as session:
            for j in range(0,col_number):
                for i in range(0,row_number):
                    task = asyncio.ensure_future(put_tile(session, i, j, zoom))
                    tasks.append(task)
            await asyncio.gather(*tasks)
            

    loop = asyncio.get_event_loop()
    loop.create_task(main(loop))
    
    return contenant

In [42]:
test = get_map(43.611715,1.4566024,2,2, zoom = 12)

Tile Rez: 9783.93962050256 X,Y: 2064,1495
Tile Rez: 9783.93962050256 X,Y: 2064,1495
Tile Rez: 9783.93962050256 X,Y: 2064,1495
Tile Rez: 9783.93962050256 X,Y: 2064,1495
200
200
200
200


In [43]:
Image.fromarray(numpy.concatenate(numpy.concatenate(test, axis= 1 ), axis=1)).save("output.jpg")